In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [21]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import sys
sys.path.append('../code')
import scrape, features
import dataframe_utilities as util

# Update DB & Get Today Games

In [23]:
last_day = pd.to_datetime(pd.read_csv('../data/game_summaries.csv')['date']).max()
get_day = last_day + pd.Timedelta(days=1)
get_day

Timestamp('2024-09-27 00:00:00')

In [24]:
#home and away
#game time
#pitchers


In [25]:
import datetime

# Example of creating a datetime sobject
date_obj = datetime.date.today()
while get_day.date() < date_obj:
    links = scrape.get_game_links(get_day)
    for l in links:
        scrape.process_link(l)
    get_day += + pd.Timedelta(days=1)

In [26]:
import lxml
test_df = scrape.get_today_games()

no pitcher {'away_team_abbr': 'PIT', 'home_team_abbr': 'TOR', 'time': '7:07PM', 'away_pitcher': 'berrijo01'}


In [27]:
test_df

,away_team_abbr,home_team_abbr,time,away_pitcher,home_pitcher,date
0,CIN,CHC,2:20PM,ashcrgr01,assadja01,2024-05-31
1,STL,PHI,6:40PM,mikolmi01,nolaaa01,2024-05-31
2,TBR,BAL,7:05PM,civalaa01,suareal01,2024-05-31
3,PIT,TOR,7:07PM,berrijo01,NaN,2024-05-31
4,DET,BOS,7:10PM,maedake01,houckta01,2024-05-31
5,WSN,CLE,7:10PM,corbipa01,bibeeta01,2024-05-31
6,TEX,MIA,7:10PM,urenajo01,sanchsi01,2024-05-31
7,ARI,NYM,7:10PM,montgjo01,severlu01,2024-05-31
8,OAK,ATL,7:20PM,searsjp01,lopezre01,2024-05-31
9,MIN,HOU,8:10PM,lopezpa01,blancro01,2024-05-31


# Process Stats for Today's Games

## Merge test and train dfs

In [28]:
test_df['is_night_game'] = True
test_df['time_numeric'] = pd.to_numeric(test_df['time'].str[:1], errors='coerce')

# Set 'is_night_game' to False where 'time_numeric' is less than 5 and not NaN
test_df.loc[(test_df['time_numeric'] < 5) & (test_df['time_numeric'].notna()), 'is_night_game'] = False

# Set 'is_night_game' to True where 'time_numeric' is NaN (indicating non-numeric original entries)
test_df.loc[test_df['time_numeric'].isna(), 'is_night_game'] = True
test_df['is_night_game'][test_df['time'].str[1:2].isin(['0','1'])] = True #for 10,11 PM games
test_df.drop(columns='time', inplace=True)

In [29]:
test_df['is_test'] = True
test_df['home_team_win']=np.nan
test_df['game_id'] = test_df.home_team_abbr + test_df.date.astype('str').str.replace('-','') + '0'
test_df.shape

(15, 10)

In [30]:
df = features.get_game_df()
df['is_test'] = False
df.shape

(935, 9)

In [31]:
df = pd.concat([df,test_df])
df = df.sort_values(by='date').reset_index(drop=True)
df.shape

(950, 10)

In [32]:
df.columns

Index(['game_id', 'home_team_abbr', 'away_team_abbr', 'date', 'is_night_game',
       'home_team_win', 'home_pitcher', 'away_pitcher', 'is_test',
       'time_numeric'],
      dtype='object')

## Add Features

In [33]:
df = features.add_trueskill_ratings(df)
df = features.add_rest_durations(df)
df.shape

(950, 19)

In [34]:
df.drop(columns=["is_test", "time_numeric"], inplace=True)

In [35]:
#datetime
date = pd.to_datetime(df['date'])
df['season'] = date.dt.year
df['month']=date.dt.month
df['week_num'] = date.dt.isocalendar().week
df['dow']=date.dt.weekday.astype('int')

In [36]:
df.shape

(950, 21)

In [37]:
df['dh_game_no'] = pd.to_numeric(df['game_id'].str[-1:],errors='coerce')
df['date'] = (pd.to_datetime(df['date']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') #epoch time

In [38]:
test_df['game_id']

0     CHC202405310
1     PHI202405310
2     BAL202405310
3     TOR202405310
4     BOS202405310
5     CLE202405310
6     MIA202405310
7     NYM202405310
8     ATL202405310
9     HOU202405310
10    KCR202405310
11    MIL202405310
12    LAD202405310
13    SEA202405310
14    SFG202405310
Name: game_id, dtype: object

### Add Stats

In [39]:
games = features.get_games()
batting = features.get_batting()
pitching = features.get_pitching()
pitchers = features.get_pitchers()
print(df.columns)

Index(['game_id', 'home_team_abbr', 'away_team_abbr', 'date', 'is_night_game',
       'home_team_win', 'home_pitcher', 'away_pitcher', 'home_trueskill_pre',
       'away_trueskill_pre', 'ts_diff', 'home_team_rest', 'away_team_rest',
       'home_pitcher_rest', 'away_pitcher_rest', 'team_rest_diff',
       'pitcher_rest_diff', 'season', 'month', 'week_num', 'dow',
       'dh_game_no'],
      dtype='object')


#### Rolling 10 Day Stats

In [40]:
b_stats = ['batting_avg','leverage_index_avg', 'onbase_perc', 'onbase_plus_slugging']
df = features.add_10RA_rolling(batting, df, b_stats, True,'batting')
print(df.columns)

AAA
AAA
AAA
AAA
             game_id home_team_abbr away_team_abbr        date  is_night_game  \
0    NYMSTL20240301D            STL            NYM  1709251200           True   
1    SEACOL20240301D            COL            SEA  1709251200           True   
2     OAKKC20240301D             KC            OAK  1709251200           True   
3    HOUWSH20240301D            WSH            HOU  1709251200           True   
4    CWSCHC20240301D            CHC            CWS  1709251200           True   
..               ...            ...            ...         ...            ...   
945     SFG202405310            SFG            NYY  1717113600           True   
946  ATLCWS20240627D            CWS            ATL  1719446400           True   
947  CLEMIN20240809D            MIN            CLE  1723161600           True   
948  MILCIN20240830D            CIN            MIL  1724976000           True   
949  NYMATL20240926N            ATL            NYM  1727308800           True   

     home_t

In [41]:
pitching['SO_batters_faced'] = pitching['so'] / pitching['batters_faced']
pitching['H_batters_faced'] = pitching['h'] / pitching['batters_faced']
pitching['BB_batters_faced'] = pitching['bb'] / pitching['batters_faced']

# create rolling stat
b_stats = ['earned_run_avg','SO_batters_faced','H_batters_faced','BB_batters_faced']
df = features.add_10RA_rolling(pitching, df, b_stats, True, 'team_pitching')

AAA
AAA
AAA
AAA
             game_id home_team_abbr away_team_abbr        date  is_night_game  \
0    NYMSTL20240301D            STL            NYM  1709251200           True   
1    SEACOL20240301D            COL            SEA  1709251200           True   
2     OAKKC20240301D             KC            OAK  1709251200           True   
3    HOUWSH20240301D            WSH            HOU  1709251200           True   
4    CWSCHC20240301D            CHC            CWS  1709251200           True   
..               ...            ...            ...         ...            ...   
945     SFG202405310            SFG            NYY  1717113600           True   
946  ATLCWS20240627D            CWS            ATL  1719446400           True   
947  CLEMIN20240809D            MIN            CLE  1723161600           True   
948  MILCIN20240830D            CIN            MIL  1724976000           True   
949  NYMATL20240926N            ATL            NYM  1727308800           True   

     home_t

In [42]:
pitchers['earned_run_avg'] = pd.to_numeric(pitchers['earned_run_avg'], errors='coerce')
pitchers['SO_batters_faced'] = pitchers['so'] / pitchers['batters_faced']
pitchers['H_batters_faced'] = pitchers['h'] / pitchers['batters_faced']
pitchers['BB_batters_faced'] = pitchers['bb'] / pitchers['batters_faced']

# create rolling stat
b_stats = ['earned_run_avg','SO_batters_faced','H_batters_faced','BB_batters_faced']
df = features.add_10RA_rolling(pitchers, df, b_stats, False, 'pitcher')

             game_id home_team_abbr away_team_abbr        date  is_night_game  \
0    NYMSTL20240301D            STL            NYM  1709251200           True   
1    SEACOL20240301D            COL            SEA  1709251200           True   
2     OAKKC20240301D             KC            OAK  1709251200           True   
3    HOUWSH20240301D            WSH            HOU  1709251200           True   
4    CWSCHC20240301D            CHC            CWS  1709251200           True   
..               ...            ...            ...         ...            ...   
945     SFG202405310            SFG            NYY  1717113600           True   
946  ATLCWS20240627D            CWS            ATL  1719446400           True   
947  CLEMIN20240809D            MIN            CLE  1723161600           True   
948  MILCIN20240830D            CIN            MIL  1724976000           True   
949  NYMATL20240926N            ATL            NYM  1727308800           True   

     home_team_win home_pit

In [43]:
print("GAMES:", df['date'])

GAMES: 0      1709251200
1      1709251200
2      1709251200
3      1709251200
4      1709251200
          ...    
945    1717113600
946    1719446400
947    1723161600
948    1724976000
949    1727308800
Name: date, Length: 950, dtype: int64


In [44]:
print(df.columns)

Index(['game_id', 'home_team_abbr', 'away_team_abbr', 'date', 'is_night_game',
       'home_team_win', 'home_pitcher', 'away_pitcher', 'home_trueskill_pre',
       'away_trueskill_pre', 'ts_diff', 'home_team_rest', 'away_team_rest',
       'home_pitcher_rest', 'away_pitcher_rest', 'team_rest_diff',
       'pitcher_rest_diff', 'season', 'month', 'week_num', 'dow', 'dh_game_no',
       'home_batting_batting_avg_10RA', 'home_batting_leverage_index_avg_10RA',
       'home_batting_onbase_perc_10RA',
       'home_batting_onbase_plus_slugging_10RA',
       'away_batting_batting_avg_10RA', 'away_batting_leverage_index_avg_10RA',
       'away_batting_onbase_perc_10RA',
       'away_batting_onbase_plus_slugging_10RA',
       'batting_batting_avg_10RA_diff', 'batting_leverage_index_avg_10RA_diff',
       'batting_onbase_perc_10RA_diff',
       'batting_onbase_plus_slugging_10RA_diff',
       'home_team_pitching_earned_run_avg_10RA',
       'home_team_pitching_SO_batters_faced_10RA',
       'home_

In [45]:
df.shape

(950, 58)

#### Games Stats

In [46]:
df = features.game_stats(games,df)

In [47]:
df.shape

(950, 81)

#### Season Stats

In [48]:
batting_stats = ['a', 'ab', 'bb', 'h', 'pa', 'po', 'r', 'rbi', 'so', 'batting_avg',
             'leverage_index_avg', 'onbase_perc', 'onbase_plus_slugging', 'pitches', 
             're24_bat', 'slugging_perc', 'strikes_total', 'wpa_bat', 'wpa_bat_neg', 
             'wpa_bat_pos']
df = features.add_season_rolling(batting, df, batting_stats, True,'batting')
df.shape

(950, 221)

In [49]:
pitching_stats = ['bb', 'er', 'h', 'hr', 'ip', 'r', 'so', 'batters_faced',
               'earned_run_avg', 'game_score', 'inherited_runners',
               'inherited_score', 'inplay_fb_total', 'inplay_gb_total', 'inplay_ld',
               'inplay_unk', 'leverage_index_avg', 'pitches', 're24_def',
               'strikes_contact', 'strikes_looking', 'strikes_swinging',
               'strikes_total', 'wpa_def','SO_batters_faced','H_batters_faced',
                'BB_batters_faced']
df = features.add_season_rolling(pitching, df, pitching_stats, True,'team_pitching')
df.shape

(950, 410)

In [50]:
if 'away_pitcher' not in df.columns:
    df['away_pitcher'] = 'Unknown'
else:
    df['away_pitcher'].fillna('Unknown', inplace=True)

In [51]:
df = features.add_season_rolling(pitchers, df, pitching_stats, False,'pitcher')
df.shape

(950, 599)

## Cleanup

In [52]:
df = util.fix_na(df, False)

In [53]:
print(df['home_team_win'])

0      0.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
945    0.0
946    0.0
947    0.0
948    0.0
949    0.0
Name: home_team_win, Length: 950, dtype: float64


# Generate Predictions

In [54]:
import pickle
import xgboost as xgb
model = xgb.XGBClassifier()
model.load_model("team_wins_model.json")
filtered_df = df[df['game_id'].isin(test_df['game_id'])]
X_test = filtered_df.drop(columns=[
    'home_team_win', 'home_team_abbr', 'away_team_abbr', 'game_id',
    'home_pitcher', 'away_pitcher', 'home_team_season', 'away_team_season'
])
# Assuming 'model' is your XGBClassifier and 'df' is your DataFrame
# First, get the list of feature names used by the model
model_features = model.get_booster().feature_names

# Ensure the DataFrame only contains the required features and in the correct order
df_for_prediction = X_test[model_features]



In [55]:
filtered_df

,game_id,home_team_abbr,away_team_abbr,date,is_night_game,home_team_win,home_pitcher,away_pitcher,home_trueskill_pre,away_trueskill_pre,...,pitcher_pitches_diff,pitcher_re24_def_diff,pitcher_strikes_contact_diff,pitcher_strikes_looking_diff,pitcher_strikes_swinging_diff,pitcher_strikes_total_diff,pitcher_wpa_def_diff,pitcher_SO_batters_faced_diff,pitcher_H_batters_faced_diff,pitcher_BB_batters_faced_diff
931,TOR202405310,TOR,PIT,1717113600,True,0.0,Unknown,berrijo01,18.922097,13.931929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932,SFG202405310,SFG,NYY,1717113600,True,0.0,hicksjo03,stromma01,25.000000,38.819339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,SEA202405310,SEA,LAA,1717113600,True,0.0,woobr01,soriajo02,37.259119,19.279773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
934,PHI202405310,PHI,STL,1717113600,True,0.0,nolaaa01,mikolmi01,41.983870,6.958836,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
935,NYM202405310,NYM,ARI,1717113600,True,0.0,severlu01,montgjo01,20.090419,25.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
936,MIL202405310,MIL,CHW,1717113600,True,0.0,myersto01,feddeer01,23.359422,25.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,MIA202405310,MIA,TEX,1717113600,True,0.0,sanchsi01,urenajo01,25.603174,41.542786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,CLE202405310,CLE,WSN,1717113600,True,0.0,bibeeta01,corbipa01,23.864882,25.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,KCR202405310,KCR,SDP,1717113600,True,0.0,wachami01,ceasedy01,25.000000,25.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,HOU202405310,HOU,MIN,1717113600,True,0.0,blancro01,lopezpa01,22.949825,31.557105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
model_features

['is_night_game',
 'home_trueskill_pre',
 'away_trueskill_pre',
 'ts_diff',
 'home_team_rest',
 'away_team_rest',
 'home_pitcher_rest',
 'away_pitcher_rest',
 'team_rest_diff',
 'pitcher_rest_diff',
 'season',
 'month',
 'week_num',
 'dow',
 'dh_game_no',
 'home_batting_batting_avg_10RA',
 'home_batting_leverage_index_avg_10RA',
 'home_batting_onbase_perc_10RA',
 'home_batting_onbase_plus_slugging_10RA',
 'away_batting_batting_avg_10RA',
 'away_batting_leverage_index_avg_10RA',
 'away_batting_onbase_perc_10RA',
 'away_batting_onbase_plus_slugging_10RA',
 'batting_batting_avg_10RA_diff',
 'batting_leverage_index_avg_10RA_diff',
 'batting_onbase_perc_10RA_diff',
 'batting_onbase_plus_slugging_10RA_diff',
 'home_team_pitching_earned_run_avg_10RA',
 'home_team_pitching_SO_batters_faced_10RA',
 'home_team_pitching_H_batters_faced_10RA',
 'home_team_pitching_BB_batters_faced_10RA',
 'away_team_pitching_earned_run_avg_10RA',
 'away_team_pitching_SO_batters_faced_10RA',
 'away_team_pitching_H_

In [57]:
df_for_prediction

,is_night_game,home_trueskill_pre,away_trueskill_pre,ts_diff,home_team_rest,away_team_rest,home_pitcher_rest,away_pitcher_rest,team_rest_diff,pitcher_rest_diff,...,pitcher_pitches_diff,pitcher_re24_def_diff,pitcher_strikes_contact_diff,pitcher_strikes_looking_diff,pitcher_strikes_swinging_diff,pitcher_strikes_total_diff,pitcher_wpa_def_diff,pitcher_SO_batters_faced_diff,pitcher_H_batters_faced_diff,pitcher_BB_batters_faced_diff
931,True,18.922097,13.931929,4.990168,24,24,23,30,0,-7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932,True,25.000000,38.819339,-13.819339,30,24,30,30,6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,True,37.259119,19.279773,17.979345,24,24,30,30,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
934,True,41.983870,6.958836,35.025034,24,24,30,30,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
935,True,20.090419,25.000000,-4.909581,24,30,30,30,-6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
936,True,23.359422,25.000000,-1.640578,24,30,30,30,-6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,True,25.603174,41.542786,-15.939611,23,24,30,30,-1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,True,23.864882,25.000000,-1.135118,24,30,30,30,-6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,True,25.000000,25.000000,0.000000,30,30,30,30,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,True,22.949825,31.557105,-8.607280,24,24,30,30,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Now you can use this DataFrame to make predictions
predictions = model.predict(df_for_prediction)
proba = model.predict_proba(df_for_prediction)


In [59]:
predictions

array([0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1])

In [60]:
proba

array([[0.56390417, 0.4360958 ],
       [0.54457253, 0.45542747],
       [0.49669397, 0.50330603],
       [0.49786282, 0.5021372 ],
       [0.5098    , 0.49019998],
       [0.4937448 , 0.5062552 ],
       [0.5886803 , 0.41131967],
       [0.4791836 , 0.5208164 ],
       [0.48330522, 0.5166948 ],
       [0.5165391 , 0.48346093],
       [0.64111024, 0.35888976],
       [0.5524291 , 0.44757092],
       [0.46362942, 0.5363706 ],
       [0.5411922 , 0.45880786],
       [0.46241146, 0.53758854]], dtype=float32)

In [61]:
import pickle
import xgboost as xgb
model = xgb.XGBRegressor()
model.load_model("total_score_model.json")
filtered_df = df[df['game_id'].isin(test_df['game_id'])]
X_test = filtered_df.drop(columns=[
    'home_team_abbr', 'away_team_abbr', 'game_id',
    'home_pitcher', 'away_pitcher', 'home_team_season', 'away_team_season'
])
# Assuming 'model' is your XGBClassifier and 'df' is your DataFrame
# First, get the list of feature names used by the model
model_features = model.get_booster().feature_names

# Ensure the DataFrame only contains the required features and in the correct order
df_for_prediction = X_test[model_features]


score_predictions = model.predict(df_for_prediction)


In [62]:
score_predictions

array([8.808601, 9.274237, 8.775832, 8.956861, 8.360405, 8.957781,
       9.032788, 8.728699, 8.949787, 9.852047, 8.583364, 8.133214,
       8.767318, 9.075979, 9.101548], dtype=float32)

In [63]:
selected_abbr = np.where(np.array(predictions) == 1, test_df['home_team_abbr'], test_df['away_team_abbr'])

# Print the result
print(selected_abbr)

['CIN' 'STL' 'BAL' 'TOR' 'DET' 'CLE' 'TEX' 'NYM' 'ATL' 'MIN' 'SDP' 'CHW'
 'LAD' 'LAA' 'SFG']


In [64]:
import json
json_predictions = []

# Loop through the DataFrame and associated predictions
for index, row in test_df.iterrows():
    # Create a dictionary for each prediction following the specified structure
    prediction = {
        "id": str(index),  # Convert index to string to match 'id: string' type
        "home_team": row['home_team_abbr'],
        "away_team": row['away_team_abbr'],
        "ml_pred": selected_abbr[index],
        "ml_conf": str(max(proba[index])),  # Convert float to string to match 'ml_conf: string'
        "ou_pred": str(score_predictions[index]),  # Placeholder values as specified
        "ou_conf": "0"   # Placeholder values as specified
    }
    json_predictions.append(prediction)

# Convert the list of dictionaries to JSON
json_output = json.dumps(json_predictions, indent=2)
print(json_output)

[
  {
    "id": "0",
    "home_team": "CHC",
    "away_team": "CIN",
    "ml_pred": "CIN",
    "ml_conf": "0.56390417",
    "ou_pred": "8.808601",
    "ou_conf": "0"
  },
  {
    "id": "1",
    "home_team": "PHI",
    "away_team": "STL",
    "ml_pred": "STL",
    "ml_conf": "0.54457253",
    "ou_pred": "9.274237",
    "ou_conf": "0"
  },
  {
    "id": "2",
    "home_team": "BAL",
    "away_team": "TBR",
    "ml_pred": "BAL",
    "ml_conf": "0.50330603",
    "ou_pred": "8.775832",
    "ou_conf": "0"
  },
  {
    "id": "3",
    "home_team": "TOR",
    "away_team": "PIT",
    "ml_pred": "TOR",
    "ml_conf": "0.5021372",
    "ou_pred": "8.956861",
    "ou_conf": "0"
  },
  {
    "id": "4",
    "home_team": "BOS",
    "away_team": "DET",
    "ml_pred": "DET",
    "ml_conf": "0.5098",
    "ou_pred": "8.360405",
    "ou_conf": "0"
  },
  {
    "id": "5",
    "home_team": "CLE",
    "away_team": "WSN",
    "ml_pred": "CLE",
    "ml_conf": "0.5062552",
    "ou_pred": "8.957781",
    "ou_conf":

In [39]:
test_df = test_df.sort_values(by=['date','game_id']).reset_index(drop=True)
pred_df = df[df.is_test][['away_pitcher', 'away_team_abbr', 'home_pitcher', 'home_team_abbr']]
pred_df['home'] = pred_df['home_team_abbr']
pred_df['away'] = pred_df['away_team_abbr']
pred_df.drop(columns=['home_team_abbr','away_team_abbr'], inplace=True)
pred_df['xgb_proba']= proba
pred_df['xgb_winner']=pred_df.home
pred_df['xgb_winner'][~pred]=pred_df.away

NameError: name 'proba' is not defined

In [40]:
# get daily odds from covers.com
import requests
from bs4 import BeautifulSoup as bs
html = requests.get('https://www.covers.com/sports/mlb/matchups').text
soup = bs(html)
games = []
for s in soup.findAll('div',{'class':'cmg_matchup_game_box cmg_game_data'}):
    g = {}
    g['home'] = s['data-home-team-shortname-search']
    g['home_odds'] = s['data-game-odd']
    
    if g['home']=='SD':g['home']='SDP'
    if g['home']=='KC':g['home']='KCR'
    if g['home']=='SF':g['home']='SFG'
    if g['home']=='WAS':g['home']='WSN'
    if g['home']=='TB':g['home']='TBR'
    
    games.append(g)
odds = pd.DataFrame(games)

# merge in the odds
pred_df = pd.merge(left=pred_df, right=odds, on='home', how='left')
pred_df['home_odds']=pd.to_numeric(pred_df['home_odds'], errors='coerce')
# pred_df['online_odds'][pred_df.xgb_probability<0.5] = -pred_df['online_odds'] #convert odds to pred winner odds (not home team odds)

In [41]:
# online proba
#https://www.bettingexpert.com/en-au/learn/understanding-betting-odds/how-to-convert-odds
pred_df['online_proba'] = -pred_df['home_odds']/(-pred_df['home_odds']+100)
pred_df['online_proba'][pred_df['home_odds']>0] = 100/(pred_df['home_odds']+100)

# Confidence
pred_df['confidence'] = np.abs(pred_df['xgb_proba']-0.5)+.5
online_conf = np.abs(pred_df['online_proba']-0.5)+.5

pred_df['conf_diff'] = pred_df['confidence'] - online_conf
pred_df['conf_diff'][(pred_df['xgb_proba']>.5)&(pred_df['online_proba']<.5)] = 'Contrary'
pred_df['conf_diff'][(pred_df['xgb_proba']<.5)&(pred_df['online_proba']>.5)] = 'Contrary'

KeyError: 'xgb_proba'

In [ ]:
# merge in team names
teams = pd.read_csv("../data/teams.csv")
pred_df = pd.merge(left=pred_df, right=teams, 
                   left_on='xgb_winner',right_on='Abbr',
                   how='left')

pred_df['pred_winner'] = pred_df['Team']
pred_df.drop(columns=['xgb_winner','Abbr','Team'], inplace=True)

In [ ]:
from IPython.display import HTML
pd.options.display.float_format = '{:.3f}'.format

pred_df['conf'] = pred_df.confidence
HTML(pred_df.sort_values(by='confidence', ascending=False).to_html(index=False))

In [ ]:
print("2-Team Parlays:")
pred_df = pred_df.sort_values(by='conf', ascending=False).reset_index(drop=True)
for i in range(5):
    t1 = pred_df.iloc[i]
    t2 = pred_df.iloc[i+1]
    print(f"- {t1.pred_winner}, {t2.pred_winner}  \t{t1.conf*t2.conf: .3f}")